# Dynamic memory allocation and deallocation

<div class="alert alert-block alert-info">
    You can find all of the C programs in this notebook in the subdirectory containing this notebook:
    <code>./src/allocating_deallocating_memory</code>
</div>

An object having allocated storage duration is one where the programmer fully controls the lifetime
of the object. To create such an object, the programmer:

* requests memory for the object using a dynamic memory allocation function
* assigns a value to the object
* uses the object for as long as required
* releases memory used by the object using a dynamic memory deallocation function

Note that the responsibility to release or deallocate the memory used by an allocated object lies with the
programmer. Failure to deallocate memory used by allocated objects is one of the common errors made by programmers.

Requesting memory is usually performed by using one of the three standard memory allocation functions declared
in the header file `<stdlib.h>`:

* `malloc`
* `calloc`
* `realloc`

Each of the memory allocation functions returns a pointer to the start of the newly allocated block of memory.
The returned memory can then be accessed using the returned pointer.

When a block of allocated memory is no longer needed the programmer should release the memory using the
function `free` passing in a pointer to the start of the allocated block of memory.

<div class="alert alert-block alert-info">
    C11 and C23 introduced the functions <code>aligned_alloc</code>, <code>free_sized</code>, and
    <code>free_aligned_size</code> that are beyond the scope of this notebook.
</div>

## `malloc`

The `malloc` function allocates a block of memory of a specified size and returns a pointer to the start of the
newly allocated memory block:

```c
void *malloc(size_t size);
```

`size` is the number of bytes of memory requested. If `size == 0` is true, then the returned pointer
must not be dereferenced (the C standard says the value of the returned pointer is implementation
defined) but it may be passed to the `free` function.
The returned pointer may be stored in any pointer type variable without casting. The newly allocated
block of memory is not initialized; it may contain any sequence of bits.

A null pointer is returned if `malloc` fails.

The following example uses `malloc` to allocate blocks of memory suitable for storing one object of various
types:

In [ ]:
// malloc_obj.c

#include <stdio.h>
#include <stdlib.h>

struct point2 {
    double x;
    double y;
};

int main(void) {
    // char
    char *c = malloc(1);                 // lifetime of char object starts here
    *c = 'a';
    printf("c points at : %c\n", *c);
    free(c);                             // lifetime of char object ends here
    
    // int
    int *i = malloc(sizeof(int));        // lifetime of int object starts here
    *i = 99;
    printf("i points at : %d\n", *i);
    free(i);                             // lifetime of int object ends here
    
    // double
    double *d = malloc(sizeof(double));  // lifetime of double object starts here
    *d = -1.5;
    printf("d points at : %f\n", *d);
    free(d);                             // lifetime of int object ends here
    
    // struct
    struct point2 *s = malloc(sizeof(struct point2));   // lifetime of struct point2 object starts here
    s->x = 0.5;
    s->y = -9.9;
    printf("s points at : (%f, %f)\n", s->x, s->y);
    free(s);                                            // lifetime of struct point2 object starts here
    
    return 0;
}

To allocate an array of length $n$ simply multiply the `sizeof` the element type by $n$:

In [ ]:
// malloc_arr.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

struct point2 {
    double x;
    double y;
};

int main(void) {
    // array of char (a string)
    size_t n = 8;
    char *c = malloc(n);
    strcpy(c, "CISC220");
    printf("c : %s\n", c);
    free(c);
    
    // array of int
    n = 3;
    int *i = malloc(n * sizeof(int));
    i[0] = 1;
    i[1] = 10;
    i[2] = 100;
    for (size_t j = 0; j < n; j++) {
        printf("i[%lu] : %d\n", j, i[j]);
    }
    free(i);
    
    // double
    double *d = malloc(n * sizeof(double));
    d[0] = -1;
    d[1] = -10;
    d[2] = -100;
    for (size_t j = 0; j < n; j++) {
        printf("d[%lu] : %f\n", j, d[j]);
    }
    free(d);
    
    // struct
    struct point2 *s = malloc(n * sizeof(struct point2));
    s[0].x = 0.5;    s[0].y = -9.9;
    s[1].x = 5.0;    s[1].y = -99.9;
    s[2].x = 50.0;   s[2].y = -999.9;
    for (size_t j = 0; j < n; j++) {
        printf("s[%lu] : (%f, %f)\n", j, s[j].x, s[j].y);
    }
    free(s);
    
    return 0;
}

A function that must create a new array and return it must dynamically allocate the array and return a
pointer to the first element of the array:

In [ ]:
// return_arr.c

#include <stdio.h>
#include <stdlib.h>

// Returns a pointer to the first element of an array of int having capacity len
int* intarray(size_t len) {
    if (len == 0) {
        len = 1;
    }
    int *arr = malloc(len);              // lifetime of array pointed at by arr starts here
    arr[0] = -99;                        // write in a test value for illustration purposes
    return arr;
}

int main(void) {
    int *a = intarray(10);
    printf("a[0] = %d\n", a[0]);
    free(a);                             // lifetime of array ends here
    return 0;
}

An alternative approach would be to require the caller to pass a pointer-to-a-pointer to the function:

In [ ]:
// return_arr2.c

#include <stdio.h>
#include <stdlib.h>

// Caller passes a pointer to a pointer that stores the result of malloc
void intarray(size_t len, int **arr) {
    if (len == 0) {
        len = 1;
    }
    int *buf = malloc(len);              // lifetime of array pointed at by arr starts here
    buf[0] = -99;                        // write in a test value for illustration purposes
    *arr = buf;
}

int main(void) {
    int *a;
    intarray(10, &a);
    printf("a[0] = %d\n", a[0]);
    free(a);                             // lifetime of array ends here
    return 0;
}

The pointer-to-a-pointer syntax may be confusing. `malloc` returns a pointer which must be stored in a variable
having a pointer type (in this case `int *` because we want an array of `int`). Recall that a C function can
change the value of a variable specified by the caller only when the caller passes a pointer to the variable:

In [ ]:
// pass_by_value.c

#include <stdio.h>

void bad_addone(double x) {
    x = x + 1;
}

void good_addone(double *x) {
    *x = *x + 1;
}

int main(void) {
    double y = 3.0;
    printf("before            : y = %f\n", y);
    
    bad_addone(y);
    printf("after bad_addone  : y = %f\n", y);
    
    good_addone(&y);
    printf("after good_addone : y = %f\n", y);
    
    return 0;
}

In `return_arr2.c`, the caller wants to set the pointer-to-int variable `a` to point at the first element of a
dynamically allocated array by calling the function `intarray`. The function can set the value of `a` only
if it receives a pointer to `a`. The type of `a` is:

```c
int *
```

or pointer-to-int. A pointer to `a` must have the type:

```c
int **
```

or pointer-to-pointer-to-int which is the type of the parameter `arr`:

```c
void intarray(size_t len, int **arr) 
```

To call the function, the caller requires the address of their variable `a` which is obtained using the
address-of operator `&`:

```c
intarray(10, &a);
```

Inside the function, `arr` is a pointer-to-pointer-to-int. To assign a value to the caller's pointer-to-int object,
we have to dereference `arr`:

```c
*arr = buf;
```

### What to do if a null pointer is returned

In principle, the memory allocation functions can fail in which case a null pointer is returned.
The generally cited advice when using any of the allocation functions is that the returned pointer should
always be tested:

```c
char *str = malloc(1000);
if (!str) {
    // null pointer, malloc failed
    // now what?
}
```

Advice for precisely what should happen inside the body of the `if` statement can be summarized as "It depends".

The basic line of thought is that the allocation function has failed because the operating system cannot
satisfy the memory request. If the operating system is truly out of memory, then there is little that a
user-level program can do to recover from the situation. Under such circumstances, the program should
release whatever resources it has allocated and shut down cleanly. The `exit` function declared in the header
`<stdlib.h>` can be used to cause normal program termination:

```c
void exit(int exit_code);
```

`exit` causes any open C streams to be flushed and closed and terminates the program returning the `exit_code`
as the exit status of the program. The following use of `exit` is commonly suggested:

```c
char *str = malloc(1000);
if (!str) {
    fprintf(stderr, "your error message here");
    exit(EXIT_FAILURE);
}
```

where `EXIT_FAILURE` is a compiler-defined exit status value. The programmer can choose a different exit status
value if desired.

More sophisticated programs might attempt to handle the low memory situation differently.

<div class="alert alert-block alert-info">
    Testing for a null pointer returned by a memory allocation function has long been the standard
    guidance given to C programmers. However, Windows is the only commonly used general computing
    operating system where the memory allocation functions return NULL on failure. In all other
    commonly used operating system, the standard memory allocation functions never return NULL and
    a low memory condition will cause the operating system to terminate
    one or more memory intensive processes instead. Interested readers should refer to
    <a href="https://doi.org/10.48550/arXiv.2208.08484">the following article for details.</a>
</div>

## `calloc`

`calloc` is similar to `malloc` except that it initializes the allocated memory so that all of its bits
are zero. The syntax for calling `calloc` is slightly different than calling `malloc`.

The `calloc` function allocates a block of memory that can hold a specified number of objects of a specified size and returns a pointer to the start of the
newly allocated memory block:

```c
void *calloc(size_t num, size_t size);
```

`num` is the number of objects
`size` is the number of bytes of memory requested. If `num` $\times$ `size` is equal to zero
then the returned pointer
must not be dereferenced (the C standard says the value of the returned pointer is implementation
defined) but it may be passed to the `free` function.
The returned pointer may be stored in any pointer type variable without casting. The newly allocated
block of memory is initialized so that all of its bits are equal to zero.

A null pointer is returned if `calloc` fails.

The following example uses `calloc` to allocate blocks of memory suitable for storing one object of various
types:

In [ ]:
// calloc_obj.c

#include <stdio.h>
#include <stdlib.h>

struct point2 {
    double x;
    double y;
};

int main(void) {
    // char
    char *c = calloc(1, 1);                 // lifetime of char object starts here
    *c = 'a';
    printf("c points at : %c\n", *c);
    free(c);                                // lifetime of char object ends here
    
    // int
    int *i = calloc(1, sizeof(int));        // lifetime of int object starts here
    *i = 99;
    printf("i points at : %d\n", *i);
    free(i);                                // lifetime of int object ends here
    
    // double
    double *d = calloc(1, sizeof(double));  // lifetime of double object starts here
    *d = -1.5;
    printf("d points at : %f\n", *d);
    free(d);                                // lifetime of int object ends here
    
    // struct
    struct point2 *s = calloc(1, sizeof(struct point2));   // lifetime of struct point2 object starts here
    s->x = 0.5;
    s->y = -9.9;
    printf("s points at : (%f, %f)\n", s->x, s->y);
    free(s);                                              // lifetime of struct point2 object starts here
    
    return 0;
}

The following program uses `calloc` to allocate an array of `unsigned int`:

In [ ]:
// return_arr_calloc.c

#include <stdio.h>
#include <stdlib.h>

// Returns a pointer to the first element of an array of unsigned int having capacity len
unsigned int* uintarray(size_t len) {
    if (len == 0) {
        len = 1;
    }
    unsigned int *arr = calloc(len, sizeof(unsigned int));  // lifetime of array pointed at by arr starts here
    return arr;
}

int main(void) {
    unsigned int *a = uintarray(10);
    printf("a[0] = %u\n", a[0]);         // guaranteed to be zero
    free(a);                             // lifetime of array ends here
    return 0;
}

## `realloc`

On occassion, a programmer may find it necessary to increase or decrease the capacity of a previously 
allocated block of memory. For example, a programmer may find it necessary to increase the capacity of
a dynamically allocated array.
The `realloc` function reallocates a previously allocated block of memory:

```c
void *realloc(void *ptr, size_t new_sz);
```

`ptr` is a pointer to the start of a valid block of memory that was previously allocated by `malloc`, `calloc`,
or `realloc` (valid means that the block of memory has not been `free`d). If `ptr` is `NULL` then the function
behaves the same as `malloc(new_sz)`.

`new_sz` is the size of the reallocated block of memory.  If `new_sz` is zero, then the returned pointer
must not be dereferenced.

The returned pointer points at the start of the reallocated block of memory, or is `NULL` if reallocation
failed. On successfuly reallocation, the reallocated memory contains the contents of the memory originally
pointed at by `ptr` that fit into the newly allocated block. Any extra space in the newly allocated
is uninitialized. `ptr` is no longer a valid pointer (must not be dereferenced) and should not be `free`d.

If the returned pointer is `NULL`, then `ptr` remains a valid pointer and the memory pointed at by `ptr` 
remains unmodified.

Conceptually, `realloc` behaves as though it performs the following steps:

1. allocates a new block of memory of `new_sz` bytes
2. copies up to `new_sz` bytes of memory starting at `ptr` into the the newly allocated block of memory
3. `free`s the block of memory pointed at by `ptr`
4. returns a pointer to the start of the newly allocated block of memory

Depending on the compiler and operating system, the actual implementation of `realloc` may differ from the
steps described above. For example, `realloc` may be able to expand or shrink the size of the memory
block in place instead of having to allocate a new block of memory.

The following example uses `realloc` to expand the capacity of a string:

In [ ]:
// realloc_str.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(void) {
    char *str = malloc(5);
    strcpy(str, "CISC");
    printf("str = %s\n", str);
    
    char *tmp = realloc(str, 8);
    if (!tmp) {
        // realloc failed, clean up and exit
        free(str);
        exit(EXIT_FAILURE);
    }
    // tmp contains the characters in str
    printf("tmp = %s\n", tmp);
    
    // reassign str to point at reallocated block of memory
    str = tmp;
    
    // use the extra memory
    strcat(str, "220");
    printf("tmp = %s\n", tmp);
    
    // always free allocated memory when it is no longer needed
    free(str);
    
    return 0;
}

## `free`

The `free` function releases previously allocated memory:

```c
void free(void *ptr);
```

`ptr` may be a null pointer or a valid pointer that was returned by `malloc`, `calloc`, or `realloc` (valid
means that the pointer points at a block of memory that has not yet been deallocated). 

Undefined behavior
results if `free` is called more than once on the same pointer value
(the double free bug). For this reason, advocates of defensive
programming suggest always assigning a value in the pointer immediately after calling `free`
(see <https://wiki.sei.cmu.edu/confluence/pages/viewpage.action?pageId=87152148>):

```c
int *p = malloc(some_value);
// code that uses p
free(p);
p = NULL;
```